<h1 align="center"> pyLDPC: Images Coding & Decoding Module Construction</h1>

## update:  03/27/16 - v.0.8.0

<b><font color="green"> Since version 0.8: Coding and decoding functions adapt the images data to the matrix's size: no conditions are required  </font></b> 

<br>


This notebook introduces the sub-module Images of pyLDPC: ldpc_images.
It explains what each function does and how to use it. If you're more interested in using the module, you may follow the <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Images.ipynb?flush_cache=true"> User's guide: pyLDPC-Images Tutorial</a>

All the functions defined below can be called py importing the sub-module ldpc_images:

```python
e.g :
from  pyldpc import ldpc_images
ldpc_images.Gray2Bin(....)
```

We'll consider 2 types of images in this study:

- Grayscale images: Images whean read by cv2.imread(), are seen as 2D arrays. Each pixel is a uint8 number (0-255). 
- RGB Images: Images that are seen as 3D arrays. Each pixel is an RGB array of 3 uint8 numbers ( [R,G,B] ). 

<font color="blue"><h2> Outline:</h2></font> 
<font color="blue"><h2> --------------------------------------------------</h2></font> 

**I. Images conversion RGB,Gray <=> Binary.**

**II. Images Coding & Decoding:**
 
**III. Application**
<font color="blue"><h2> --------------------------------------------------</h2></font> 


We'll consider 2 types of images in this study:

- Grayscale images: Images whean read by cv2.imread(), are seen as 2D arrays. Each pixel is a uint8 number (0-255). 
- RGB Images: Images that are seen as 3D arrays. Each pixel is an RGB array of 3 uint8 numbers ( [R,G,B] ). 

To code and decode images, we'll have to "binarize" them first:

- Graysale images are binarized (using Gray2bin function that we'll present later) into 3D arrays where each uint8 number is written as a 8-bits binary array. Therefore, Coding needs a 8-rows coding matrix. The image obtained is shaped: (width, height, 8).
- RGB images are binarized pixel by pixel (using RGB2bin): each color is transformed to an 8-bits binary array, then the 3 arrays obtained of each pixel are gathered into a binary 24-bits array. The image obtained is shaped (width, height, 8).


In [110]:
import warnings
import numpy as np
import scipy
from IPython.display import display, HTML
import pyldpc 
import cv2 
from time import time
from math import ceil 

# I - RGB, Gray - Binary conversion
First 2 fundamental functions that transform INT <=> Binary Array

In [25]:
def int2bitarray(N,k):
    """
    Changes array's base from int (base 10) to binary (base 2)
    
    Parameters:
    
    N: int N 
    k: Width of the binary array you would like to change N into. N must not be greater than 2^k - 1. 
    
    >> Examples: int2bitarray(6,3) returns [1, 1, 0]
                 int2bitarray(6,5) returns [0, 0, 1, 1,0]
                 int2bitarray(255,8) returns [1, 1, 1, 1, 1, 1, 1, 1]
                 int2bitarray(255,10) returns [0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


    """

    binary_string = bin(N) 
    length = len(binary_string)
    bitarray = np.zeros(k, 'int')
    for i in range(length-2):
        bitarray[k-i-1] = int(binary_string[length-i-1])

    return bitarray

def bitarray2int(bitarray):
    
    """ Changes array's base from binary (base 2) to int (base 10).
    
    Parameters:
    
    bitarray: Binary Array.

    >> Examples: bitarray2int([1, 1, 0]) returns 6
                 bitarray2int([0, 0, 1, 1,0]) returns 6
                 bitarray2int([1, 1, 1, 1, 1, 1, 1, 1]) returns 255
                 

    
    """
    
    bitstring = "".join([str(i) for i in bitarray])
    
    return int(bitstring,2)

## I-1 - Binarizing grayscale images:
Gray2Bin and Bin2Gray transform 2D-graysclae images to 3D-binary arrays

For a grayscale image:
<img src="Equations/Construction19.png">

<i>Gray2Bin</i> returns:

<img src="Equations/Construction20.png">

<i>Bin2Gray </i> does the reverse operation.
   

In [26]:
def Gray2Bin(img):
    """ Puts a GrayScale Image on a binary form 
    
    Parameters:
    
    img_array: 2-D array of a grayscale image (no 3rd dimension)
    
    returns:
    
    3-D img_array in a binary form, each pixel uint8 is transformed to an 8-bits array
    
    
    >>> Example:  the grayscale (2x2) image [[2, 127],    
                                             [255, 0]]
                                        
    will be conveterted to the (2x2x8) binary image:   [[[0, 0, 0, 0, 0, 0, 1, 0],[0, 1, 1, 1, 1, 1, 1, 1]],
                                                        [[1, 1, 1, 1, 1, 1, 1, 1],[0, 0, 0, 0, 0, 0, 0, 0]]]
                                                       
                                                       
    """
    if not len(img.shape)==2:
        raise ValueError('{} must have 2 dimensions. Make sure it\'s a grayscale image.')
        
    height,width = img.shape
    
    img_bin = np.zeros(shape=(height,width,8),dtype=int)
    
    for i in range(height):
        for j in range(width):
            img_bin[i,j,:] = int2bitarray(img[i,j],8)
            
    return img_bin
    

In [27]:
def Bin2Gray(img_bin):
    
    """ Puts a 8-bits binary Image to uint8
    
    Parameters:
    
    img_array: 3-D array (height, width, 8)
    
    returns:
    
    2-D img_array in grayscale
    
    >>> Example:  the (2x2x8) binary image:   [[[0, 0, 0, 0, 0, 0, 1, 0],[0, 1, 1, 1, 1, 1, 1, 1]],
                                                          [[1, 1, 1, 1, 1, 1, 1, 1],[0, 0, 0, 0, 0, 0, 0, 0]]] 
                                            
                                        
    will be conveterted to the (2x2) uint8 image [[2, 127],
                                                  [255, 0]]
    
    """
    height,width,k = img_bin.shape
    img_grayscale = np.zeros(shape=(height,width),dtype=np.uint8)
    
    for i in range(height):
        for j in range(width):
            img_grayscale[i,j] = bitarray2int(img_bin[i,j,:])
            
    return img_grayscale

## I-2- Binarizing RGB Images:
RGB2Bin and Bin2RGB transform 3D colorful images to 3D-binary arrays.

In [28]:
def RGB2Bin(img):
    """ Puts an RGB Image on a binary form 
    
    Parameters:
    
    img_array: 3-D array of an RGB image ( 3rd dimension = 3)
    
    returns:
    
    3-D img_array in a binary form, each pixel is transformed to an 24-bits binary array.
    
    
    >>> Example:  the grayscale (2x1x3) image [[[2, 127,0]],
                                               [[255, 0,1]]]
                                        
    will be conveterted to the (2x1x24) binary image:   
[[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]]
                                                       
                                                       
    """
    height,width,depth = img.shape

    if not depth==3:
        raise ValueError('{}\'s 3rd dimension must be equal to 3 (RGB). Make sure it\'s an RGB image.')
      
    
    img_bin = np.zeros(shape=(height,width,24),dtype=int)
    
    for i in range(height):
        for j in range(width):
            R = int2bitarray(img[i,j,0],8)
            G = int2bitarray(img[i,j,1],8)
            B = int2bitarray(img[i,j,2],8)

            img_bin[i,j,:] = np.concatenate((R,G,B))
            
    return img_bin
    

In [29]:
def Bin2RGB(img_bin):
    
    """ Puts a 24-bits binary Image to 3xuint8 (RGB)
    
    Parameters:
    
    img_array: 3-D array (height, width, 24)
    
    returns:
    
    3-D img_array in RGB (height, width, 3)
    
    >>> Example:  the (2x1x24) binary image:   
    
   [[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
   [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]]
  
                                        
    will be conveterted to the (2x1x3) RGB image :
    
    [[[2, 127,0]],
    [[255, 0,1]]]

    """
    
    height,width,depth = img_bin.shape
    img_rgb = np.zeros(shape=(height,width,3),dtype=np.uint8)
    
    for i in range(height):
        for j in range(width):
            R = bitarray2int(img_bin[i,j,:8])
            G = bitarray2int(img_bin[i,j,8:16])
            B = bitarray2int(img_bin[i,j,16:])

            img_rgb[i,j] = np.array([R,G,B],dtype=np.uint8)
            
    return img_rgb

# II -  Coding and decoding

**Note**: <b><font color="green"> If you'are fimiliar with prior versions &lt; v.0.7, rowbyrow and pixel coding functions are now the same. The functions adapt the images data to the matrix's size: no conditions are required  </font></b>

 ImageCoding codes a binary image and adds a AWGN with specified SNR. SNR: Signal-Noise Ratio, SNR = 10log(1/variance) in decibels. G has to be systematic to visualize the noisy image before decoding. 
 

<b><font color="red"> Since v.0.8 Images Coding and Decoding functions return noisy and decoded images in binary format. Apply Bin2RGB or Bin2Gray afterward.  </font></b>


### Coding:

In [392]:
def ImageCoding(tG,img_bin,snr):
    
    """ 
    
    CAUTION: SINCE V.0.7 Image coding and decoding functions TAKE TRANSPOSED CODING MATRIX tG. USE
    SCIPY.SPARSE.CSR_MATRIX FORMAT (IN H AND G) TO SPEED UP CALCULATIONS. USE A SYSTEMATIC CODING MATRIX WITH 
    CodingMatrix_systematic. THEN USE SCIPY.SPARSE.CSR_MATRIX()
    
    --------
    
    Codes a binary image (Therefore must be a 3D-array). The image is reshaped so as to match the Coding and decoding matrices.
    
    - The last line of the coded image stores the initial image's shape so as to be able to construct the decoded image again 
    via a reshape. 
    - n is the length of a codeword. Then a gaussian noise N(0,snr) is added to the codeword.
    
    Remember SNR: Signal-Noise Ratio: SNR = 10log(1/variance) in decibels of the AWGN used in coding.
 
    ImageCoding returns  a tuple: the reshaped coded image, and the noisy (binary) image.
    
    Parameters:

    tG: Transposed Coding Matrix G - must be systematic. See CodingMatrix_systematic.
    img_bin: 3D-array of a binary image.
    SNR: Signal-Noise Ratio, SNR = 10log(1/variance) in decibels of the AWGN used in coding.    
    
    Returns:
    (default): Tuple:  coded_img, noisy_img
    
    """
    n,k = tG.shape
    height,width,depth = img_bin.shape
    
    if n>=100 and not type(tG)==scipy.sparse.csr_matrix:
        warnings.warn("Using scipy.sparse.csr_matrix format is highly recommanded when computing coding and decoding with large matrices to speed up calculations.")
        
    if n>=100 and not (tG[:k,:]==np.identity(k)).all():
        raise ValueError("G must be Systematic if n>=100 (for later decoding, solving tG.tv = tx for images has a O(n^3) complexity.)")
       

    ratio = height*width*depth//k
    rows = ceil(height*width*depth/k)
    rest = height*width*depth%k
        
    img_bin_reshaped = np.zeros((rows,k),dtype=int)
    
    img_bin_reshaped[:ratio,:] = img_bin.flatten()[:ratio*k].reshape(ratio,k)
    if rest >0:
        img_bin_reshaped[-1,:rest] = img_bin.flatten()[-rest:]

    coded_img = np.zeros(shape=(rows+1,n))
    coded_img[rows,0:3]=height,width,depth
    
    for i in range(rows):
        coded_img[i,:] = Coding(tG,img_bin_reshaped[i,:],snr)
        
    noisy_img = (coded_img[:rows,:k]<0).astype(int).flatten()[:height*width*depth].reshape(height,width,depth)

    
    return coded_img,noisy_img


### Decoding 

ImageDecoding requires also Coding Matrix G in ordrer to solve v.G = x where x is the codeword element decoded by the function itself if G is not systematic. Otherwise, the first k bits of the codeword are exactly v.


When v is found for each codeword, the decoded image is returned in a binary form. 

<br>

<font color="blue"> ImageDecoding uses a slightly different version of decoding functions (pyldpc.Decoding_BP, pyldpc.Decoding_logBP) where some "static" parameters related to H are passed to the decoding functions in order to speed up computing. These optimization changes are hidden so that the user will not suffer from radical syntax changes in new releases.</font>
    

In [393]:
def ImageDecoding(tG,H,img_coded,snr,max_iter=1,log=1):
    
    """ 
        
    CAUTION: SINCE V.0.7 ImageDecoding TAKES TRANSPOSED CODING MATRIX tG INSTEAD OF G.IF N>100, USE SCIPY.SPARSE.CSR_MATRIX 
    FORMAT (IN H AND G) TO SPEED UP CALCULATIONS. 
    
    --------
    Image Decoding Function. Taked the 3-D binary coded image where each row is a codeword n-bits array and decodes 
    every one of them. Needs H to decode and tG to solve tG.tv = tx where x is the codeword element decoded by the function
    itself. When v is found for each codeword, the decoded image is returned in a binary form. 
    
    Parameters: 
    
    tG: Transposed Coding Matrix ( IF N>100, SCIPY.SPARSE.CSR_MATRIX FORMAT RECOMMANDED )
    H: Parity-Check Matrix (Decoding matrix).( IF N>100, SCIPY.SPARSE.CSR_MATRIX FORMAT RECOMMANDED)

    img_coded: binary coded image returned by the function ImageCoding.
    
    snr: Signal-Noise Ratio: SNR = 10log(1/variance) in decibels of the AWGN used in coding.
    
    max_iter: (optional, default =1), number of iterations of decoding. 
    
    log: (optional, default = True), if True, Full-log version of BP algorithm is used. 

    
    """
    
    n,k = tG.shape
    size = img_coded[-1,0:3].astype(int)
    img_coded = img_coded[:-1,:]
    rows,N = img_coded.shape
    depth = size[-1]
    
    if N!=n:
        raise ValueError('Coded Image must have the same number of columns as H')
        
    if depth !=8 and depth != 24:
        raise ValueError('type of image not recognized: third dimension of the binary image must be 8 for grayscale, or 24 for RGB images')

    if n>=100 and not type(tG)==scipy.sparse.csr_matrix:
        warnings.warn("Using scipy.sparse.csr_matrix format is highly recommanded when computing coding and decoding with large matrices to speed up calculations.")
        
    if n>=100 and not (tG[:k,:]==np.identity(k)).all():
        raise ValueError("G must be Systematic. Solving tG.tv = tx for images has a O(n^3) complexity.")
        
    img_decoded_bin = np.zeros(shape=(rows,k),dtype = int)

    if log:
        DecodingFunction = pyldpc.decodingfunctions.Decoding_logBP_ext
    else:
        DecodingFunction = pyldpc.decodingfunctions.Decoding_BP_ext
    
    BitsNodes = pyldpc.ldpcalgebra.BitsAndNodes(H)

    for i in range(rows):
        decoded_vector = DecodingFunction(H,BitsNodes,img_coded[i,:],snr,max_iter)
        img_decoded_bin[i,:] = decoded_vector[:k]
            

        img_decoded = img_decoded_bin.flatten()[:np.prod(size)].reshape(size)
 
    return img_decoded

### Bonus: Bit Error Rate: 
Function that will give us an idea about how accurate our decoding is by comparing the original and the decoded images bit by bit. It returns the rate of false bits over all bits.

In [34]:
def BER(original_img_bin,decoded_img_bin):
    """ 
    
    Computes Bit-Error-Rate (BER) by comparing 2 binary images.
    The ratio of bit errors over total number of bits is returned.
    
    """
    if not original_img_bin.shape == decoded_img_bin.shape:
        raise ValueError('Original and decoded images\' shapes don\'t match !')
        
    height, width, k = original_img_bin.shape 
    
    
    errors_bits = sum(abs(original_img_bin-decoded_img_bin).reshape(height*width*k))
    total_bits  = np.prod(original_img_bin.shape)
    
    BER = errors_bits/total_bits 
    
    return(BER)


# III- Application :
<a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Images.ipynb?flush_cache=true"> User's guide: pyLDPC-Images Tutorial</a>